In [5]:
import pandas as pd
import numpy as np
import os.path
import glob
import random
import boto3
import shutil

In [6]:
raw = pd.read_csv('validated.tsv', sep='\t', header=0)

In [7]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1425784 entries, 0 to 1425783
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   client_id   1425784 non-null  object
 1   path        1425784 non-null  object
 2   sentence    1425778 non-null  object
 3   up_votes    1425784 non-null  int64 
 4   down_votes  1425784 non-null  int64 
 5   age         907791 non-null   object
 6   gender      910859 non-null   object
 7   accent      704024 non-null   object
 8   locale      1425784 non-null  object
 9   segment     32726 non-null    object
dtypes: int64(2), object(8)
memory usage: 108.8+ MB


In [8]:
accent = raw[raw.accent.notna()]
accent.accent.value_counts()

us                351472
england           118401
indian             73030
canada             48453
australia          46951
scotland           12676
newzealand         11281
other              10505
ireland             9233
african             8066
philippines         4158
singapore           2967
hongkong            2750
malaysia            1685
wales               1550
bermuda              643
southatlandtic       203
Name: accent, dtype: int64

In [9]:
accent['sentence_len'] = accent['sentence'].apply(lambda x: len(x.split(" ")))
accent['is_asian'] = accent['accent'].apply(lambda x: x in ('philippines','singapore','hongkong','malaysia', 'indian'))
accent['us_eu'] = accent['accent'].apply(lambda x: x in ('us','england','canada','australia', 'scotland', 'newzealand', 'ireland', 'wales'))

<ipython-input-9-c189eb3cb503>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accent['sentence_len'] = accent['sentence'].apply(lambda x: len(x.split(" ")))
<ipython-input-9-c189eb3cb503>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accent['is_asian'] = accent['accent'].apply(lambda x: x in ('philippines','singapore','hongkong','malaysia', 'indian'))
<ipython-input-9-c189eb3cb503>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [53]:
all_asian = accent[accent['is_asian']]
non_asian = accent[accent['us_eu']]

all_asian = all_asian[all_asian['sentence_len'] > 1]
non_asian = non_asian[non_asian['sentence_len'] > 1]

In [54]:
asian_sample = all_asian.groupby('accent').sample(150)
non_asian_sample = non_asian.groupby('accent').sample(100)

In [55]:
print("Asian_Sample: " + str(len(asian_sample)))
print("Non_Asian_Sample: " + str(len(non_asian_sample)))

Asian_Sample: 750
Non_Asian_Sample: 800


In [56]:
asian_sample.sentence_len.mean()

9.389333333333333

In [57]:
non_asian_sample.sentence_len.mean()

9.43125

In [58]:
asian_sample.to_csv('asian_sample.csv')
non_asian_sample.to_csv('non_asian_sample.csv')

In [59]:
asia_group = asian_sample['path'].tolist()
nonasia_group = non_asian_sample['path'].tolist()

In [60]:
def getclips(filelist, sample_size):
    counter = 0
    choosen = []
    working_copy = filelist
    while counter < sample_size:
        if len(working_copy) == 0:
            counter = 999
        else:
            ran_num = random.randint(0,len(working_copy)-1)
            if (os.path.isfile("clips/{}".format(working_copy[ran_num]))):
                choosen.append(working_copy[ran_num])
                del working_copy[ran_num]
                counter += 1
            else:
                del working_copy[ran_num]
    return choosen

In [61]:
asia_clips = getclips(asia_group, 500)
nonasia_clips = getclips(nonasia_group, 500)

In [63]:
len(asia_clips)

500

In [64]:
len(nonasia_clips)

500

In [65]:
for clip in asia_clips:
    original = 'C:/Users/Quok Juin/Desktop/cv-corpus-7.0-2021-07-21/en/clips/{}'.format(clip)
    target = 'C:/Users/Quok Juin/Desktop/cv-corpus-7.0-2021-07-21/en/asian/{}'.format(clip)

    shutil.copyfile(original, target)

for clip in nonasia_clips:
    original = 'C:/Users/Quok Juin/Desktop/cv-corpus-7.0-2021-07-21/en/clips/{}'.format(clip)
    target = 'C:/Users/Quok Juin/Desktop/cv-corpus-7.0-2021-07-21/en/non_asian/{}'.format(clip)

    shutil.copyfile(original, target)

In [67]:
asian_sample[asian_sample.path.isin(asia_clips)].to_csv('asian_selected.csv')
non_asian_sample[non_asian_sample.path.isin(nonasia_clips)].to_csv('non_asian_selected.csv')